<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Twitter - Get members of list
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/template.ipynb" target="_parent">
<img src="https://img.shields.io/badge/-Open%20in%20Naas-success?labelColor=000000&logo="/>
</a><br><br><a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=&template=template-request.md&title=Tool+-+Action+of+the+notebook+">Template request</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=&template=bug_report.md&title=[ERROR]+Tool+/+Folder+Action+of+the+notebook+">Bug report</a>

**Tags:** #twitter #tweepy #pandas #twitterautomation #twitterlistmembers #snippet

**Author:** [Kaushal Krishna](https://www.linkedin.com/in/kaushal-krishna-a48959153)

**Description:** This notebook removes a single member from member list

<u>References:</u>
- https://developer.twitter.com/en/docs/twitter-api/lists/list-members/introduction

## Input

### Import libraries

In [ ]:
import tweepy
import pandas as pd
import naas

### Setup Variables
-> Get your bearer token by applying in twitter developer website https://developer.twitter.com/en/docs/authentication/oauth-2-0/bearer-tokens  
-> Get your Oauth 1.0 token by applying into twitter developer website with Read and Write Permission https://developer.twitter.com/en/docs/authentication/oauth-1-0a     
-> For the user name supply the display name (Twitter Handle) without '@' i.e.  "JupyterNaas" for Naas.ai    
-> Please consult the tweepy docs for more help on authentication https://docs.tweepy.org/en/stable/authentication.html

**The bearer_token is for Oauth 2.0 user lookup and consumer_key, consumer_secret ,access_token, access_token_secret are for Oauth 1.0 get_user(by username parameter) list lookup(even private ones),list_member lookup  list add member and remove member**

In [ ]:
#inputs
tw_bearer_token = naas.secret.get("TWITTER_BEARER_TOKEN") or 'AAAAAAAAAAAAAAAAAAAAAMQhdgEAAAAAD0EI4sREei02us...'
tw_consumer_key=naas.secret.get("TWITTER_CONSUMER_KEY") or 'xxxxxxxxxxxxxxx'
tw_consumer_secret=naas.secret.get("TWITTER_CONSUMER_SECRET") or 'xxxxxxxxxxxxxxxxx'
tw_access_token=naas.secret.get("TWITTER_ACCESS_TOKEN") or 'xxxxxxxxxxxxxxxxxxxxxxx'
tw_access_token_secret=naas.secret.get("TWITTER_ACCESS_TOKEN_SECRET") or 'xxxxxxxxxxxxxx'
user_name = naas.secret.get("TWITTER_NAME") or 'JupyterNaas'
list_name=naas.secret.get("TWITTER_LIST_NAME") or 'xxxxxxxxxxxxxxxxxxxxx'
member_username=naas.secret.get("TWITTER_MEMBER_NAME") or 'JupyterNaas'


#outputs
csv_output = 'output.csv'

# Setup Functions

In [ ]:
def find_list_id(list_name,user_name,client,user_auth=True):
    list_id=None
    user = client.get_user(username=user_name,user_auth=user_auth)
    user_id=user.data['id']
    twitter_list=client.get_owned_lists(id=user_id,user_auth=user_auth)
    for row in twitter_list.data:
        if(row['name']==list_name):
            list_id=row['id']
            break
    return list_id

In [ ]:
def get_members_of_list(client,list_name,user_name,print_choice=False,user_auth=True):

    data_dict={
        "Username":[],
        "Name":[]
              }

    user = client.get_user(username=user_name,user_auth=user_auth)
    user_id=user.data['id']
    twitter_list=client.get_owned_lists(id=user_id,user_auth=user_auth)

    for row in twitter_list.data:
        if(row['name']==list_name):
            list_id=row['id']
            list_members=client.get_list_members(id=list_id,user_auth=user_auth)
            for member in list_members.data:
                data_dict['Username'].append(member['username'])
                data_dict['Name'].append(member['name'])
    
    return pd.DataFrame(data_dict)

In [ ]:
def find_member_id(member_username,list_id,client,user_auth=True):
    member_id=None
    list_members=client.get_list_members(id=list_id,user_auth=user_auth)
    for member in list_members.data:
        if(member['username']==member_username):
            member_id=member['id']
            break
    return member_id

## Model

### Connect to client

In [ ]:
try:
    client_1=tweepy.Client(bearer_token=tw_bearer_token)
    client_2=tweepy.Client(consumer_key=tw_consumer_key,consumer_secret=tw_consumer_secret,
                           access_token=tw_access_token,access_token_secret=tw_access_token_secret)
except:
    print("Failure in Authentication.")

In [ ]:
#init result
list_id=None
member_id=None

In [ ]:
list_id=find_list_id(list_name=list_name,user_name=user_name,client=client_2)

In [ ]:
member_id=find_member_id(member_username=member_username,list_id=list_id,client=client_2)

In [ ]:
prev_df=get_members_of_list(client=client_2,list_name=list_name,user_name=user_name)

In [ ]:
res=client_2.remove_list_member(id=list_id,user_id=member_id,user_auth=True)

In [ ]:
res_df=get_members_of_list(client=client_2,list_name=list_name,user_name=user_name)

## Output

# Display Output

In [ ]:
prev_df

In [ ]:
res_df

### Save result in csv

In [ ]:
res_df.to_csv(csv_output, index=False)